# Generating resulting graphs for MARGI files

In [1]:
from google import colab
colab.drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install pyranges

In [3]:
import os
import tarfile
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import re
import pyranges as pr
import pandas as pd


In [4]:
cd /content/drive/MyDrive/Z.Chen/

/content/drive/MyDrive/Z.Chen


In [ ]:
from coexp_update import get_coexp_network_updated

In [6]:
get_coexp_network_updated("LINC00607", "./iMARGI_extracted/GSM4006840_HUVEC_control_iMARGI.bedpe.gz", 5)

/content/drive/MyDrive/Z.Chen/coexp_update.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contained["container_len"] = contained["End_b"] - contained["Start_b"]


KeyboardInterrupt: 

In [ ]:
df = pd.read_csv("./iMARGI_extracted/GSM4006842_HUVEC_H_T7d_iMARGI.bedpe.gz", sep="\t", header=None)

In [ ]:
df.shape

In [ ]:
df.head(3)

In [ ]:
gene_names = pd.read_csv('./genes_df.csv')

In [ ]:
gene_names[gene_names['gene_name']=='LINC00607']

In [ ]:
df.columns = [
"RNA_chr", "RNA_start", "RNA_end",
"DNA_chr", "DNA_start", "DNA_end",
"name", "score", "RNA_strand", "DNA_strand"
]

In [ ]:
df.shape[1]

In [ ]:
genes = gene_names[['Chromosome','Start','End','gene_name']]

In [ ]:
genes.head()

In [ ]:
df.head(2)

In [ ]:
chr2:215,611,563–215,843,535

In [ ]:
df.head()

In [ ]:
df = df.rename(columns={
    'RNA_chr':'Chromosome',
    'RNA_start':'Start',
    'RNA_end':'End'
})


In [ ]:
df = df[['Chromosome','Start','End', 'DNA_chr', 'DNA_start', 'DNA_end']]

In [ ]:
gene_names

In [ ]:
rna_annot = gene_names

In [ ]:

pr_df = pr.PyRanges(df)
pr_rna = pr.PyRanges(rna_annot)


In [ ]:
joined = pr_df.join(pr_rna, how="left")

In [ ]:
jdf = joined.df

In [ ]:
contained = jdf[(jdf["Start"] >= jdf["Start_b"]) & (jdf["End"] <= jdf["End_b"])]

In [ ]:
contained["container_len"] = contained["End_b"] - contained["Start_b"]

In [ ]:
best = (contained
        .sort_values(["Chromosome","Start","End","container_len"])
        .drop_duplicates(subset=["Chromosome","Start","End"], keep = 'first'))

In [ ]:
out = df.merge(best[["Chromosome","Start","End","gene_name"]],
               on=["Chromosome","Start","End"], how="left")

In [ ]:
out

In [ ]:
import pandas as pd
import pickle  # for protocol constant if you want

# Save
out.to_pickle("./out_pickle", protocol=pickle.HIGHEST_PROTOCOL)  # Colab path



In [ ]:
out = pd.read_pickle("./out_pickle")

In [ ]:
out.head(2)

In [ ]:
out = out.rename(columns={
    'Chromosome' : 'RNA_chr',
    'Start' : 'RNA_start',
    'End' : 'RNA_end',
    'gene_name' : 'RNA_gene_name'
})

In [ ]:
out = out.rename(columns={
    'DNA_chr' : 'Chromosome',
    'DNA_start' : 'Start',
    'DNA_end' : 'End',
})

In [ ]:
out

In [ ]:
gene_names = pd.read_csv('./genes_df.csv')

In [ ]:
gene_names

In [ ]:
out

In [ ]:
out_dna = out[['Chromosome', 'Start', 'End']]

In [ ]:
gene_names = pd.read_csv('./genes_df.csv')

In [ ]:
gene_names.head()

In [ ]:
pr_df = pr.PyRanges(out_dna)
pr_dna = pr.PyRanges(gene_names)

In [ ]:
joined = pr_df.join(pr_dna, how="left")

In [ ]:
jdf = joined.df

In [ ]:
contained = jdf[(jdf['Start']>= jdf['Start_b']) & (jdf['End']<= jdf['End_b'])]

In [ ]:
contained['container_len'] = contained['End_b'] - contained['Start_b']

In [ ]:
best = (contained
        .sort_values(['Chromosome', 'Start', 'End', 'container_len'])
        .drop_duplicates(subset=['Chromosome', 'Start', 'End'], keep='first'))

In [ ]:
best

In [ ]:
out

In [ ]:
out2 = out.merge(best[['Chromosome', 'Start', 'End', 'gene_name']],
               on=['Chromosome', 'Start', 'End'], how='left')

In [ ]:
out2 = out2.rename(columns={
    'gene_name' : 'DNA_gene_name'
})

In [ ]:
out2

In [ ]:
out2.to_pickle('./complete_df.csv')

In [ ]:
df = out2

In [ ]:
query_df = df[df['RNA_gene_name'] == "LINC00607"][['RNA_gene_name', 'DNA_gene_name']]

In [ ]:
query_df

In [ ]:
vc = query_df.value_counts().reset_index(name='count')

In [ ]:
vc

In [ ]:
import networkx as nx
query = 'LINC00607'
freq = 5
G = nx.Graph()
G.add_node(query)

for _, row in vc.iterrows():
    dna_label = row['DNA_gene_name']
    G.add_node(dna_label, count=row['count'])
    G.add_edge(query, dna_label, weight=row['count'])

plt.figure(figsize=(14, 12))
pos = nx.spring_layout(G, k=0.6, seed=42)

node_sizes = []
node_colors = []

for node in G.nodes():
    if node == query:
        node_sizes.append(800)
        node_colors.append('red')
    else:
        count = G.nodes[node].get("count", 1)
        node_sizes.append(100 + count * 10)
        # Highlight if count > frequency
        if count > freq:
            node_colors.append('orange')  # Highlight color
        else:
            node_colors.append('skyblue')

In [ ]:
plt.figure(figsize=(14, 12))
pos = nx.spring_layout(G, k=0.6, seed=42)

nx.draw(
    G, pos,
    with_labels=True,
    node_size=node_sizes,
    node_color=node_colors,
    edge_color='gray',
    font_size=8
)

plt.title(f"{query} Interaction Network (Highlighting count > {freq})", fontsize=16)
plt.axis('off')
plt.show()